# Projeto 1 - Ciência dos Dados

Nome: Rodrigo Santos Neves Rocha

Nome: Pietro Zanga Neto

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
nltk.download('rslp')
stemmer = nltk.stem.RSLPStemmer()

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Rsngamer\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\Rsngamer\Documents\Jupyter Notebook\CDados\Projeto 1\github


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [8]:
filename = 'Tesla.xlsx'

In [10]:
dados_train = pd.read_excel(filename, sheet_name='Treinamento')
dados_test= pd.read_excel(filename,sheet_name='Teste')

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O produto escolhido se refere à marca de automóveis Tesla, produtora de veículos elétricos. Os tweets foram classificados como relevantes ou irrelevantes a partir de certos requisitos:
1. O tweet se refere a um automóvel Tesla diretamente(ex. "joão comprou um Tesla Model X novo ontem")
2. O tweet se refere à empresa Tesla Motors(ex. "comprei ações da Tesla outro dia")
3. O tweet não se refere exclusivamente a Nikola Tesla ou seus feitos(ex. "quero montar uma bobina de Tesla")

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [14]:
def cleanup(tweet):
    for palavra in tweet:
        if '@' in palavra or 'https' in palavra or "&amp" in palavra or "&gt" in palavra:
            tweet.replace(palavra,"")

    #tweet = tweet.replace("", "")    TEMPLATE
    tweet = tweet.replace(" '", "")
    tweet = tweet.replace("' ", "")
    tweet = tweet.replace("#", "")
    tweet = tweet.replace("!", " ")
    tweet = tweet.replace("?", " ")
    tweet = tweet.replace("(", " ")
    tweet = tweet.replace(")", " ")
    tweet = tweet.replace(" rt ", " ")
    tweet = tweet.replace("rt ", " ")
    tweet = tweet.replace("¥", "")
    tweet = tweet.replace("%", "")
    tweet = tweet.replace("}", "")
    tweet = tweet.replace("{", "")
    tweet = tweet.replace("|", "")
    tweet = tweet.replace("\\", "")
    tweet = tweet.replace("/", "")
    tweet = tweet.replace("&", "")
    tweet = tweet.replace(";", "")
    tweet = tweet.replace("£", "")
    tweet = tweet.replace("€", "")
    tweet = tweet.replace("•", "")
    tweet = tweet.replace("*", "")
    tweet = tweet.replace("=", "")
    tweet = tweet.replace("~", "")
    tweet = tweet.replace("", "")

    tweet = tweet.lower()
    tweet = tweet.split()
    
    clean_tweet = []
    for palavra in tweet:
        clean_tweet.append(stemmer.stem(palavra))

    return clean_tweet

In [15]:
total_relevante = dados_train['Treinamento'][dados_train['Relevante'] == 0].count()
total_irrelevante = dados_train['Treinamento'][dados_train['Relevante'] == 1].count()

total = total_relevante + total_irrelevante


In [16]:
palavras_relevantes = []
palavras_irrelevantes = []

frequencia_relevantes = {}
frequencia_irrelevantes = {}

n_relevante = 0
n_irrelevante = 0

i = 0
for relevancia in dados_train['Relevante']:
    clean_tweet = cleanup(dados_train['Treinamento'][i])
    for palavra in clean_tweet:
        if relevancia == 1:
            if palavra not in palavras_irrelevantes:
                palavras_irrelevantes.append(palavra)
                frequencia_irrelevantes[palavra] = 1
            else:
                frequencia_irrelevantes[palavra] += 1
            n_irrelevante += 1
            
        else:
            if palavra not in palavras_relevantes:
                palavras_relevantes.append(palavra)
                frequencia_relevantes[palavra] = 1
            else:
                frequencia_relevantes[palavra] += 1
            n_relevante += 1
            
    i += 1
    
n_total = n_relevante + n_irrelevante

In [20]:
def p_compara(p_0, p_1):
    if p_0 > p_1:
        return 0
    else:
        return 1

In [21]:
def p_calc(tweet, palavras, n_palavras):
    prob = 0
    total = n_palavras
    tweet_limpo = cleanup(tweet)
    for palavra in tweet_limpo:
        frequencia = 1
        if palavra in palavras:
            frequencia += palavras[palavra]
        prob += frequencia/(total + n_total)
    return prob

In [22]:
lista_previsao = []
for tweet in dados_test['Teste']:
    prob_1 = p_calc(tweet, frequencia_irrelevantes, n_irrelevante)
    prob_0 = p_calc(tweet, frequencia_relevantes, n_relevante)
    lista_previsao.append(p_compara(prob_0, prob_1))

dados_test['Previsão'] = lista_previsao

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
acertos = 0
indice = 0
for avaliacao in dados_test['Relevante']:
    if avaliacao == dados_test['Previsão'][indice]:
        acertos += 1
    indice += 1

proporcao_acertos = acertos/len(dados_test)
porcentagem_acertos = proporcao_acertos*100
print(f"Acertos: {acertos}")
print(len(dados_test))
print(128/200)
print(f"Classificações corretas(porcentagem): {porcentagem_acertos:.2f}%")    

Acertos: 128
200
0.64
Classificações corretas(porcentagem): 64.00%


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**